In [1]:
from emoji import demojize
from nltk.tokenize import TweetTokenizer

# intialize tokenizer

tokenizer = TweetTokenizer()

# define token normalization function

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

# define tweet normalization function

def normalizeTweet(tweet):
    tweet = tweet.replace("<url>", "HTTPURL")
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())

In [2]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"The current working directory is: {current_directory}")

The current working directory is: /home/russell_pang/Desktop/ETH/FS24/CIL/project


In [2]:
# Import necessary libraries for training and testing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import os
import torch
from torch import tensor
import torch.nn.functional as F
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from trl import SFTTrainer
from transformers import TrainingArguments, Trainer
from typing import Tuple
from datasets import load_dataset, Dataset

#model_name = #'finiteautomata/bertweet-base-sentiment-analysis' #'siebert/sentiment-roberta-large-english'
#model_name = 'vinai/bertweet-large'
#model_name = 'Neo-11/tweet_sentiment'
#model_name = 'Ibrahim-Alam/finetuning-xlnet-base-cased-on-tweet_sentiment_binary'
#  
# For this approach we use a pre-trained model which is based on RoBERTa

model_name = 'tweettemposhift/sentiment-sentiment_small_random3_seed2-bertweet-large'
output_dir = "./text_classification/datasets/"

CLASSES = 2


data = pd.read_csv(output_dir + "preprocessed_train.csv")
data_sample = data.sample(frac=1, random_state=42)

In [3]:
#model_name = 'Ibrahim-Alam/finetuning-xlnet-base-cased-on-tweet_sentiment_binary' 

# Load the pretrained model as well as the tokenizer

model = AutoModelForSequenceClassification.from_pretrained(model_name)#, num_labels=CLASSES)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = model.config.max_position_embeddings
print("model parameters:" + str(sum(p.numel() for p in model.parameters())))

model parameters:355361794


In [7]:
for param in model.roberta.parameters():
    param.requires_grad = False

In [4]:

model = model.cuda()
#text = "congratulations! you are the best"
text = "I really hate that"

encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
encoded_input["input_ids"] = encoded_input["input_ids"].cuda()
#encoded_input["token_type_ids"] = encoded_input["token_type_ids"].cuda()
encoded_input["attention_mask"] = encoded_input["attention_mask"].cuda()
output = model(**encoded_input)
scores = output[0][0].detach().cpu().numpy()
scores = F.softmax(torch.tensor(scores), dim=0).numpy()
scores.tolist()

{'input_ids': tensor([[   0,  100,  269, 4157,   14,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


[0.9933673739433289, 0.006632662378251553]

In [5]:
labels = data_sample["label"].tolist()
labels = [0 if x == 0 else (1 if CLASSES == 2 else 2) for x in labels] 
# convert labels to one hot vectors
labels = np.eye(CLASSES)[labels]
labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [6]:
# Prepare the training and validation sets

train_data,val_data, train_labels, val_labels = train_test_split(data_sample["text"], labels, test_size=1, random_state=42)
dataset = Dataset.from_list([{'text': text, 'labels': label} for text, label in zip(train_data, train_labels)])
val_dataset = Dataset.from_list([{'text': text, 'labels': label} for text, label in zip(val_data, val_labels)])

def tokenize_function(examples):
    return tokenizer(examples['text'])

dataset = dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
dataset

Map:   0%|          | 0/2499999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 2499999
})

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [8]:
#!wandb login ->here_your_api<-

wandb: Appending key for api.wandb.ai to your netrc file: /home/russell_pang/.netrc


In [9]:
torch.cuda.empty_cache()

In [10]:
# Transfer learning part - define the training arguments and the trainer

from transformers import TrainerCallback, TrainerState, TrainerControl,training_args


# custom callback class for accuracy evaluation during the training
class CustomCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        # Assuming the evaluation dataset has 'labels' and 'predictions' fields
        eval_dataloader = kwargs['eval_dataloader']
        model = kwargs['model']
        tokenizer = kwargs['tokenizer']
        
        model.eval()
        correct = 0
        total = 0
        
        for batch in eval_dataloader:
            inputs = batch['input_ids'].to(args.device)
            labels = batch['labels'].to(args.device)
            
            with torch.no_grad():
                outputs = model(inputs)
                predictions = torch.argmax(outputs.logits, dim=-1)
            
            labels = torch.argmax(labels, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
        
        accuracy = correct / total
        print(f"Evaluation Accuracy: {accuracy:.4f}")

# initialization of the trainer

bs = 16
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=val_dataset,
    args=TrainingArguments(
        per_device_train_batch_size=bs,
        gradient_accumulation_steps=256//bs,
        warmup_steps=10,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim=training_args.OptimizerNames.ADAMW_TORCH, learning_rate=4e-5,
        # optim=training_args.OptimizerNames.LION,learning_rate=1e-5,
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        num_train_epochs=2,
        report_to="wandb",
        group_by_length=True,
        evaluation_strategy="steps",
        eval_steps=300,
        save_steps=0.25,
    ),
    callbacks=[CustomCallback()],
)

# Run the training

model.config.problem_type = "multi_label_classification"
trainer_stats = trainer.train()

/home/russell_pang/anaconda3/envs/cil_new/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: russellpang0503 (russellpang0503-eth-z-rich). Use `wandb login --relogin` to force relogin


  0%|          | 0/19530 [00:00<?, ?it/s]

{'loss': 0.8419, 'grad_norm': 19.232431411743164, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.8085, 'grad_norm': 18.96207618713379, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6499, 'grad_norm': 11.58893871307373, 'learning_rate': 1.2e-05, 'epoch': 0.0}
{'loss': 0.5423, 'grad_norm': 7.165821075439453, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.0}
{'loss': 0.5296, 'grad_norm': 10.728065490722656, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 0.437, 'grad_norm': 7.189696311950684, 'learning_rate': 2.4e-05, 'epoch': 0.0}
{'loss': 0.3543, 'grad_norm': 4.837492942810059, 'learning_rate': 2.8e-05, 'epoch': 0.0}
{'loss': 0.3931, 'grad_norm': 8.277115821838379, 'learning_rate': 3.2000000000000005e-05, 'epoch': 0.0}
{'loss': 0.4007, 'grad_norm': 7.405435562133789, 'learning_rate': 3.6e-05, 'epoch': 0.0}
{'loss': 0.3427, 'grad_norm': 4.59221887588501, 'learning_rate': 4e-05, 'epoch': 0.0}
{'loss': 0.2741, 'grad_norm': 3.8430376052856445, 'learn

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.0009552726405672729, 'eval_runtime': 0.0177, 'eval_samples_per_second': 56.651, 'eval_steps_per_second': 56.651, 'epoch': 0.03}
Evaluation Accuracy: 1.0000
{'loss': 0.2466, 'grad_norm': 2.380704402923584, 'learning_rate': 3.940368852459017e-05, 'epoch': 0.03}
{'loss': 0.2299, 'grad_norm': 1.8227200508117676, 'learning_rate': 3.94016393442623e-05, 'epoch': 0.03}
{'loss': 0.2811, 'grad_norm': 2.4222519397735596, 'learning_rate': 3.939959016393443e-05, 'epoch': 0.03}
{'loss': 0.2431, 'grad_norm': 2.0880966186523438, 'learning_rate': 3.939754098360656e-05, 'epoch': 0.03}
{'loss': 0.2016, 'grad_norm': 1.4338990449905396, 'learning_rate': 3.9395491803278696e-05, 'epoch': 0.03}
{'loss': 0.2005, 'grad_norm': 1.3801220655441284, 'learning_rate': 3.9393442622950824e-05, 'epoch': 0.03}
{'loss': 0.2338, 'grad_norm': 1.7436527013778687, 'learning_rate': 3.939139344262295e-05, 'epoch': 0.03}
{'loss': 0.2026, 'grad_norm': 1.361409306526184, 'learning_rate': 3.938934426229509e-05, 'epo

...

{'eval_loss': 0.0002306612441316247, 'eval_runtime': 0.0122, 'eval_samples_per_second': 82.303, 'eval_steps_per_second': 82.303, 'epoch': 1.94}
Evaluation Accuracy: 1.0000
{'loss': 0.1465, 'grad_norm': 2.170577049255371, 'learning_rate': 1.2889344262295085e-06, 'epoch': 1.94}
{'loss': 0.2357, 'grad_norm': 2.640833854675293, 'learning_rate': 1.2868852459016395e-06, 'epoch': 1.94}
{'loss': 0.1474, 'grad_norm': 2.059718370437622, 'learning_rate': 1.2848360655737706e-06, 'epoch': 1.94}
{'loss': 0.1744, 'grad_norm': 2.0585997104644775, 'learning_rate': 1.2827868852459017e-06, 'epoch': 1.94}
{'loss': 0.1218, 'grad_norm': 1.6917022466659546, 'learning_rate': 1.2807377049180328e-06, 'epoch': 1.94}
{'loss': 0.2089, 'grad_norm': 2.301968574523926, 'learning_rate': 1.2786885245901639e-06, 'epoch': 1.94}
{'loss': 0.1569, 'grad_norm': 2.1202940940856934, 'learning_rate': 1.2766393442622952e-06, 'epoch': 1.94}
{'loss': 0.1655, 'grad_norm': 1.8068029880523682, 'learning_rate': 1.2745901639344262e-06,

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.0002306612441316247, 'eval_runtime': 0.013, 'eval_samples_per_second': 76.808, 'eval_steps_per_second': 76.808, 'epoch': 1.97}
Evaluation Accuracy: 1.0000
{'loss': 0.1509, 'grad_norm': 2.4466044902801514, 'learning_rate': 6.741803278688525e-07, 'epoch': 1.97}
{'loss': 0.1755, 'grad_norm': 2.263895034790039, 'learning_rate': 6.721311475409836e-07, 'epoch': 1.97}
{'loss': 0.1897, 'grad_norm': 2.191112518310547, 'learning_rate': 6.700819672131148e-07, 'epoch': 1.97}
{'loss': 0.1607, 'grad_norm': 2.0356948375701904, 'learning_rate': 6.68032786885246e-07, 'epoch': 1.97}
{'loss': 0.1422, 'grad_norm': 1.8933711051940918, 'learning_rate': 6.659836065573771e-07, 'epoch': 1.97}
{'loss': 0.2445, 'grad_norm': 2.684718370437622, 'learning_rate': 6.639344262295082e-07, 'epoch': 1.97}
{'loss': 0.2045, 'grad_norm': 2.1383185386657715, 'learning_rate': 6.618852459016394e-07, 'epoch': 1.97}
{'loss': 0.1541, 'grad_norm': 2.0571560859680176, 'learning_rate': 6.598360655737706e-07, 'epoch':

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.0002306612441316247, 'eval_runtime': 0.012, 'eval_samples_per_second': 83.542, 'eval_steps_per_second': 83.542, 'epoch': 2.0}
Evaluation Accuracy: 1.0000
{'loss': 0.2398, 'grad_norm': 2.61657452583313, 'learning_rate': 5.942622950819672e-08, 'epoch': 2.0}
{'loss': 0.1388, 'grad_norm': 1.8201292753219604, 'learning_rate': 5.737704918032787e-08, 'epoch': 2.0}
{'loss': 0.1725, 'grad_norm': 2.5389387607574463, 'learning_rate': 5.5327868852459016e-08, 'epoch': 2.0}
{'loss': 0.2128, 'grad_norm': 2.293832540512085, 'learning_rate': 5.3278688524590166e-08, 'epoch': 2.0}
{'loss': 0.1431, 'grad_norm': 2.0848817825317383, 'learning_rate': 5.122950819672131e-08, 'epoch': 2.0}
{'loss': 0.2076, 'grad_norm': 2.5288805961608887, 'learning_rate': 4.9180327868852466e-08, 'epoch': 2.0}
{'loss': 0.2054, 'grad_norm': 2.688011646270752, 'learning_rate': 4.7131147540983616e-08, 'epoch': 2.0}
{'loss': 0.1617, 'grad_norm': 2.005014419555664, 'learning_rate': 4.508196721311476e-08, 'epoch': 2.0}

In [21]:
# save the model and the tokenizer

torch.save(model, output_dir + "bertweet.pt")
torch.save(tokenizer, output_dir + "bertweet_tokenizer.pt")

In [54]:
torch.cuda.empty_cache()
#3 is the best

In [11]:
model = model.cuda()
model = model.eval()

In [12]:
text = "congratulations! you are the best"
# text = "I really hate that"

encoded_input = tokenizer(text, return_tensors='pt')
encoded_input["input_ids"] = encoded_input["input_ids"].cuda()
encoded_input["attention_mask"] = encoded_input["attention_mask"].cuda()
output = model(**encoded_input)
scores = output[0][0].detach().cpu().numpy()
scores = F.softmax(torch.tensor(scores), dim=0).numpy()
scores.tolist()

[8.851569646139978e-07, 0.9999991655349731]

In [18]:
test_df = pd.read_csv(output_dir + "preprocessed_test.csv")

#test_df["text"] = test_df["text"].apply(normalizeTweet)

In [19]:
batch_size=50
all_preds = []
with torch.no_grad():
    for i in range(0, len(test_df), batch_size):
        batch = test_df["text"][i:i+batch_size].tolist()
        tokens = tokenizer(batch, padding=True, return_tensors="pt")
        tokens = {k: v.cuda() for k, v in tokens.items()}
        output = model(**tokens)
        logits = output[0].cpu()
        scores = F.softmax(logits, dim=1)[:,(1 if CLASSES == 2 else 2)]
        all_preds.extend(scores.tolist())

In [20]:
# save the predictions

binary_predictions = [1 if prob >= 0.5 else -1 for prob in all_preds]
print(len(binary_predictions))
submission_df = pd.DataFrame({"Prediction": binary_predictions})
submission_df["Id"] = submission_df.index
submission_df["Id"] = submission_df["Id"] + 1
submission_df = submission_df[["Id", "Prediction"]]
submission_df.to_csv(output_dir + "submission_bertweet.csv", index=False)

test_probs_df = pd.DataFrame({'Probability': all_preds})
test_probs_df.to_csv(output_dir + "test_probs_bertweet.csv", index=False)

10000
